<a href="https://www.kaggle.com/code/search4/zomato-eda-archana-yadav?scriptVersionId=142098075" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

#### Name: Archana Yadav
#### Institute: AI Adventures
#### DataSet : Zomato-EDA (Bangalore)
* Dataset having 17 columns [url, address, name, online_order, book_table, rate, votes, phone, location, rest_type, dish_liked, cuisines, approx_cost(for two people), reviews_list, menu_item, listed_in(type), listed_in(city)]

#### Operations: Exploratory Data Analysis (Data Cleaning,Data Visualization),Restaurant Search by taking location,Restaurant type,Cuisine type and budget

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#### Read csv

In [ ]:
df = pd.read_csv('/kaggle/input/zomato-eda/zomato.csv')
df.head()

In [ ]:
# check data types of each columns
df.dtypes

In [ ]:
# checking null values
df.isna().sum()

In [ ]:
# checking duplicate values
df.duplicated().sum()

In [ ]:
# create a copy of main dataset for further operation
new_df = df.copy()

In [ ]:
# checking shae of dataframe
new_df.shape

In [ ]:
new_df['rest_type'].head(5)

In [ ]:
# split values by ',' using split() which returns list of values 
new_df['rest_type'] = new_df['rest_type'].str.split(', ')

In [ ]:
# as you can see the output after spliting
new_df['rest_type']

In [ ]:
# it will create a new rows for every value present in list with all other data remain as it is 
new_df = new_df.explode(['rest_type'])

# here you can see the output of explode()
new_df.rest_type.head()

In [ ]:
new_df= new_df.explode(['listed_in(type)'])

In [ ]:
new_df['listed_in(type)'].unique()

#### Rename columns

In [ ]:
new_df.rename(columns={'address':'Restro_full_address','name':'Restro_name','rate':'ratings','rest_type':'resto_type','reviews_list':'reviews','location':'city','approx_cost(for two people)':'approx_cost_for_two_people'},inplace = True)

In [ ]:
new_df.head(2)

#### Drop columns

##### droping not required columns i.e., : 'listed_in(city)', 'menu_item', 'reviews', 'url' & 'dish_liked' 

In [ ]:
new_df.drop(columns=['listed_in(city)','menu_item','reviews','url','dish_liked'], inplace=True)

In [ ]:
new_df.head(2)

In [ ]:
# after droped the columns again check the duplicated data here you can see 16378 rows are duplicated 
new_df.duplicated().sum()

In [ ]:
# drop duplicated rows
new_df.drop_duplicates(inplace = True)

#### Check null values

In [ ]:
#After droped duplicated values again check null values
new_df.isna().sum()

#### Operations on Restaurant name (Restro_name) column

In [ ]:
# number of unique values in restro_name
new_df.Restro_name.nunique()

In [ ]:
#here you can see the error value / garbbage value in the name of restaurant 
new_df.Restro_name.unique()[7300:7350]

In [ ]:
# by the observation we concluded that all garbage values looks like below strings to replaced it with empty string
new_df['Restro_name'].replace(r'[Ã\x83]|[Â\x82]|[\x92]+','',regex=True, inplace = True)

In [ ]:
#output after replacement
new_df.Restro_name.unique()[7300:7350]

##### For filling the missing values most of the Grouping is done with Restro_name and city:

##### Now why grouping with Restro_name and city ? 
    Because it might be chances that the multiple 'city' (cities) having same restro_name (Restaurant Name) and 
    the same city having multiple restaurants so you need both for grouping and replacing the null values

#### Operations on Restaurant City (city) column

In [ ]:
new_df.loc[new_df['city'].isna(),['Restro_full_address','city']]

In [ ]:
# create a group of restaurant name and address and Split the Restaurant full address by ',' and considered only of value at -2 or at -1 

address = new_df.groupby(['Restro_name','Restro_full_address']).agg({'Restro_full_address':lambda x: pd.Series.mode(x)[0]}).rename(columns={'Restro_full_address':'addrs'})
city_l = address.reset_index()
city_l['addrs']=city_l.addrs.str.split(',').apply(lambda x: x[-2] if len(x)>1 else x[-1])
city_l

In [ ]:
# match the column's data and if match found replace the respective value with missing value
def add_city(x):
    return city_l.loc[(x[0]==city_l['Restro_name']) & (x[1]==city_l['Restro_full_address']),'addrs'].values[0]

new_df.loc[new_df['city'].isna(),'city']=new_df.loc[new_df['city'].isna()].loc[:,['Restro_name','Restro_full_address']].apply(add_city,axis=1)

In [ ]:
new_df['city'].unique()

In [ ]:
new_df['city'].isna().sum()

#### Operation on Restaurant type 

In [ ]:
new_df.loc[new_df['resto_type'].isna(),['Restro_name','city','resto_type','listed_in(type)']].head(50)

In [ ]:
# Grouping with Restaurant name and city and restaurant type(listed_in(type)) by its mode and replace this value with null value of 'Resto_type'

rest = new_df.groupby(['Restro_name','city']).agg({'listed_in(type)':pd.Series.mode})
res = rest.reset_index()
res = res.explode(['listed_in(type)'])

def add_rest_type(x):
    
    ''' This function takes Restaurant name and city as a tuple where `Restro_type` is null. Then it will compare Restro_name and city 
    columns with grouped columns resceptively and return the `listed_in(type)' values'''
    
    return res.loc[(x[0]==res['Restro_name']) & (x[1]==res['city']),'listed_in(type)'].values


# Replacing null value of resto_type with add_rest_type()'s value

new_df.loc[new_df['resto_type'].isna(),'resto_type']=new_df[new_df['resto_type'].isna()].loc[:,['Restro_name','city']].apply(add_rest_type,axis=1)
    

In [ ]:
# explode multiple value checking unique value of columns
new_df = new_df.explode(['resto_type']) 

new_df.resto_type.unique()

In [ ]:
# Now you don't need listed_in(type) column for further operation so drop this column
new_df.drop('listed_in(type)',axis=1,inplace=True)

In [ ]:
# Replace the some of value of restro_type 
new_df['resto_type'] =new_df['resto_type'].astype(str).replace(['Dine-out','Cafes','Bhojanalya','Fine Dining','Desserts'],['Casual Dining','Cafe','Mess','Casual Dining','Dessert Parlor'])

In [ ]:
new_df['resto_type'].unique()

In [ ]:
# Again after dropping the listed_in(type) columns you need to check the duplicate rows if there then drop it
new_df.duplicated().sum()

In [ ]:
# Drop the duplicated rows
new_df.drop_duplicates(inplace=True)

In [ ]:
new_df.duplicated().sum()

In [ ]:
# Here you can see the resto_type column now having 0 null values
new_df.isna().sum()

#### Operation on Approx cost for two people

In [ ]:
# Last 10 data of approx_cost_for_two_people. Here you observe the data is in object type 
#that contain cost with seperated by ',' so you need to do some operation on it to convert it into float 


new_df.approx_cost_for_two_people.tail(10)

In [ ]:
# before converting it into float you need to replace the ','
new_df['approx_cost_for_two_people']= new_df['approx_cost_for_two_people'].str.replace(',','')

In [ ]:
# Convert object dtype with float by using astype()
new_df['approx_cost_for_two_people']=new_df['approx_cost_for_two_people'].astype(float)

In [ ]:
new_df[new_df['approx_cost_for_two_people'].isnull()].head()

In [ ]:
def approx_cost():
    '''This function use to replace the missing values by grouping restro_name,city and 
    approx_cost_for_two_people by calculating the mean of approx_cost_for_two_people column.
  '''
    
    x=new_df.groupby(["Restro_name",'city']).agg({'approx_cost_for_two_people':np.mean})
    x.reset_index(inplace=True)
    return x[['Restro_name','approx_cost_for_two_people']]

df1=approx_cost()

In [ ]:
# performing fill null values
for i in df1.values:
    try:
        if i[0] in list(new_df.loc[new_df['approx_cost_for_two_people'].isna(),'Restro_name']):
            a=float(i[1])
            if a>0:
                new_df.loc[new_df['Restro_name']==i[0],'approx_cost_for_two_people'] = a
    except:
        pass
new_df['Restro_name']

In [ ]:
# the above operation returns float value which may contain some more decimalpoint values so rounding up is required here
new_df['approx_cost_for_two_people']=new_df['approx_cost_for_two_people'].round(2)

In [ ]:
# you can see the rounded up values
new_df['approx_cost_for_two_people'].unique()

In [ ]:
# earilier  approx_cost_for_two_people   having  170 null values after filling the null value it is now 56 
new_df.isnull().sum()

#### Operation on Cuisines column

In [ ]:
new_df.cuisines.unique()

In [ ]:
def add_cuisines(x):
    '''This function group the restro_name,city and cuisines by mode 
    and match the columns with x (tuple having Restro_name and city) and return the value'''
    
    a= new_df.loc[new_df['cuisines'].notna()].groupby(['Restro_name','city']).agg({'cuisines':pd.Series.mode})
    l=a.reset_index()
    return l.loc[(x[0]==l['Restro_name']) & (x[1]==l['city']),'cuisines'].values

# fill missing value with appropiate data
new_df.loc[new_df['cuisines'].isnull(),'cuisines']=new_df[new_df['cuisines'].isnull()].loc[:,['Restro_name','city']].apply(add_cuisines,axis=1)

In [ ]:
new_df['cuisines']=new_df['cuisines'].str.split(', ')

In [ ]:
#explode the multiple value rows
new_df=new_df.explode('cuisines')

In [ ]:
new_df.cuisines.unique()

In [ ]:
new_df.cuisines.tail()

In [ ]:
new_df.isna().sum()

#### Operations on Ratings (ratings) column

In [ ]:
# here you might be observe the what kind of data we have so before going for further you need to make you data as per you requirement
new_df.ratings.unique()

In [ ]:
# Here 'NEW' & '-' will replace with 0 and /5 and white space with ''
new_df['ratings'].replace(['NEW','-'],0, inplace = True)
new_df['ratings'].replace(r'[\/5 | \s]','',regex=True, inplace = True)

In [ ]:
new_df.ratings.unique()

In [ ]:
new_df.loc[new_df['ratings'].isna(),['Restro_name','city','ratings']].head()

In [ ]:
new_df.loc[new_df['Restro_name']=='Bombay Kulfis',['Restro_name','city','ratings']]

In [ ]:
# Covert object dtype to float
new_df['ratings']=new_df['ratings'].astype('float')

In [ ]:
new_df.ratings.unique()

In [ ]:
def add_ratings(x):
    '''Performing grouping and match the columns with respective tuple values of parameter and return the mean value'''
    
    rate= new_df.loc[new_df['ratings'].notna()].groupby(['Restro_name','city']).agg({'ratings':'mean'}).round(1)
    rate1=rate.reset_index()
    return rate1.loc[(x[0]==rate1['Restro_name']) & (x[1]==rate1['city']),'ratings'].values

# filling null value with respective mean value
new_df.loc[new_df['ratings'].isnull(),'ratings']=new_df[new_df['ratings'].isna()].loc[:,['Restro_name','city']].apply(add_ratings,axis=1)

In [ ]:
new_df.ratings.astype(str).unique()

In [ ]:
# Replace empty list [] (this is nothing but replacement of null values) with 0 
#(why 0? = because null ratings may = 0 for this condition so instead of droping it we can repalce it by 0)

new_df.loc[new_df.astype(str)['ratings'] == '[]','ratings'] = 0

In [ ]:
# Convert object data into float type
new_df['ratings']=new_df['ratings'].astype('float')

In [ ]:
new_df.isna().sum()

In [ ]:

new_df.duplicated().sum()

In [ ]:
new_df.drop_duplicates(inplace=True)

In [ ]:
new_df.isna().sum()

#### Operation Phone column

In [ ]:
# As you observed the phone data having some irrelavent data 
new_df.loc[new_df['Restro_name']=='Jalsa',['Restro_name','phone']]

In [ ]:
# You may observe the occurrance of \r is always before \n and number of occurance for \r is 0 or more (*) and for \n 1 or more (+)
new_df['phone'] = new_df['phone'].str.split(r'\r*\n+', regex = True)

In [ ]:
# after cleaning the phone data now check for missing values
new_df.loc[new_df['phone'].isna(),['Restro_name','city','phone']].head(10)

In [ ]:
# try to filling the null data with same restro_name and city

phone_list = new_df.loc[new_df['phone'].notna()].groupby(['Restro_name','city']).agg({'phone': pd.Series.mode})
ph = phone_list.reset_index()



def add_phone(x):
    ''' This function match the restro_name and city with the value of parameter (tuple  value) and return the appropaite value'''
    
    return ph.loc[(x[0]==ph['Restro_name']) & (x[1]==ph['city']),'phone'].values

new_df.loc[new_df['phone'].isna(),'phone']=new_df[new_df['phone'].isna()].loc[:,['Restro_name','city']].apply(add_phone,axis=1)

In [ ]:
new_df.phone

In [ ]:
new_df.isna().sum()

In [ ]:
# Drop the null values
new_df.dropna(inplace=True)

### Data Visualization


##### Once EDA is done for further analysis Data Visualization is done. It gives more clear view about the dataset

In [ ]:
# Libraries for data visualization
import seaborn as sns
import matplotlib.pyplot as plt

##### 1. Restaurants delivering Online or not

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize= (6,5))
s = sns.countplot(data=new_df, x='online_order', hue='online_order')
s.set_xlabel('Online Order')
s.set_ylabel('Number of restaurants')
s.set_title('Restaurants delivering Online or not')

##### Here you can see the number of restaurant offers the online order service or not
##### approx 45000 Restaurtant  provides online order service 

##### 2. Restaurants allowing table booking or not

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize= (6,5))
s = sns.countplot(data=new_df, x='book_table', hue='book_table')
s.set_xlabel('table booking')
s.set_ylabel('Number of restaurants')
s.set_title('Restaurants allowing table booking or not')

##### Here, you can observed that the number of restaurant gives table booking service are much less then the No table booking service or you can say that the dataset having less data related to table booking service, it might be create biasness toward No table booking in model  

##### 3. Table booking Rate  vs Rating 

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize= (6,5))
s = sns.barplot(data=new_df, x='book_table',y='ratings')
s.set_xlabel('table booking option')
s.set_ylabel('ratings')
s.set_title('Table booking Rate vs Ratings')

##### By looking the above plot you can say that the ratings are more for table booking service 

##### Therefore it might be chances that people are more liking to having table booking service 

##### 4. Highest and Lowest Cities ranking by Ratings and Votes with or without Online order service and their Restaurant's Cuisines  

In [ ]:
# create new dataFrame with selected columns
pair = new_df[['Restro_name','city','resto_type','cuisines','online_order','ratings', 'votes','book_table']]

In [ ]:
# Grouped resto_type,city,cuisines,online_order by sumition of the ratings value and votes value and sort it in descending order
cuis = pair.groupby(['resto_type','city','cuisines','online_order']).agg({'ratings':'sum','votes':'sum'}).sort_values(by= ['ratings','votes'],ascending = False)

In [ ]:
# Reset the index and assign the grouped dataframe by seperating then online_order column's value (Yes and No) 

cuis.reset_index(inplace = True)
yes_order = cuis[cuis['online_order']=='Yes'].head(10)
No_order = cuis[cuis['online_order']=='No'].head(10)

In [ ]:
# assigning lowest ranking 10 values which are greater than 0
cuis_tail = cuis[cuis['ratings']>0]

# Same operation for the lowest ranking
cuis_tail.reset_index(inplace = True)
yes_order_t = cuis_tail[cuis_tail['online_order']=='Yes'].tail(10)
No_order_t = cuis_tail[cuis_tail['online_order']=='No'].tail(10)

In [ ]:
fig,axes = plt.subplots(nrows=8, ncols=1, figsize=(12,45))   
fig.subplots_adjust(bottom=0.2, left=0.2)

s = sns.barplot(data=yes_order,x= 'ratings', y='city',hue = 'cuisines',ax=axes[0])
s.set_title('Top Cities with Restaurant having Online order Services and thier cuisnes v/s Ratings')
s.set_xlabel('Ratings \n\n BTM having Restaurant with more ratings and \nmost of them serves North Indian foods with online order service\n But it has some outliers')
              
s1 = sns.barplot(data=No_order,x= 'ratings', y='city',hue = 'cuisines',ax=axes[1])
s1.set_title('Top Cities with Restaurant having No Online order Services and thier cuisnes v/s Ratings')
s1.set_xlabel('Ratings \n\n Koramangala 5th Block also having Restaurant with more ratings and \nmost of them serves North Indian foods with no online order service')


s2 = sns.barplot(data=yes_order,x= 'votes', y='city',hue= 'cuisines',ax=axes[2])
s2.set_title('Top Cities with Restaurant having Online order Services and thier cuisnes v/s Votes')
s2.set_xlabel('Votes \n\n Koramangala 5th block having Restaurant with more Votes and \nmost of them serves North Indian foods with online order service')


s3 = sns.barplot(data=No_order,x= 'votes', y='city',hue= 'cuisines',ax=axes[3])
s3.set_title('Top Cities with Restaurant having No Online order Services and thier cuisnes v/s Votes')
s3.set_xlabel('Votes \n\n\n Koramangala 5th block having Restaurant with more Votes and \nmost of them serves Cafe with no online order service \n\n---------------------------------------------')

# tail

s4 = sns.barplot(data=yes_order_t,x= 'ratings', y='city',hue = 'cuisines',ax=axes[4])
s4.set_title('Cities with Restaurant having Online order Services and thier cuisnes with lowest Ratings')
s4.set_xlabel('Ratings \n\nUlsoor having Restaurant with lowest ratings and \nthey serves Pizza & Italian with online order service')


s5 = sns.barplot(data=No_order_t,x= 'ratings', y='city',hue = 'cuisines',ax=axes[5])
s5.set_title('Cities with Restaurant having No Online order Services and thier cuisnes with lowest Ratings')
s5.set_xlabel('Ratings \n\n Bellandur having Restaurant with lowest ratings and \nthey serves Pizza & Italian with no online order service')


s6 = sns.barplot(data=yes_order_t,x= 'votes', y='city',hue= 'cuisines',ax=axes[6])
s6.set_title('Cities with Restaurant having Online order Services and thier cuisnes with lowest Votes')
s6.set_xlabel('Votes \n\n Ulsoor having Restaurant with lowest votes and \nthey serves Pizza & Italian with online order service')


s7 = sns.barplot(data=No_order_t,x= 'votes', y='city',hue= 'cuisines',ax=axes[7])
s7.set_title('Cities with Restaurant having No Online order Services and thier cuisnes with lowestvotes')
s7.set_xlabel('Votes \n\n Bellandur having Restaurant with lowest votes and \nthey serves Pizza & Italian with no online order service')



plt.tight_layout(pad=5.0)

##### By observing all the above plots concluded that if you want to open the restaurant then the best area by top ratings and votes is Koramangala 5th Block and you may go for North Indian or Cafe cuisines option.

##### You may open a restuarant in Ulsoor or Bellandur because it may having less ratings and votes in some cuisines but it may happen that they might be having less Restaurant related to some particular cuisines or people don't like their services. So if you provide some good services then might be having chances to increase your target



##### 4.1. type of Cuisines available in Koramagala and BTM

In [ ]:
cc = new_df.groupby(['city','cuisines']).agg({'cuisines':'count'}).rename(columns = {'cuisines':'count'})
cc.reset_index(inplace=True)

In [ ]:
BTM_cuisines = cc[cc['city']=='BTM'].sort_values(by='count',ascending=False)
BTM_head=BTM_cuisines.head(10)
BTM_tail=BTM_cuisines.tail(10)

In [ ]:
Kora_cuisines = cc[cc['city']=='Koramangala 5th Block'].sort_values(by='count',ascending=False)
Kora_head=Kora_cuisines.head(10)
Kora_tail=Kora_cuisines.tail(10)

In [ ]:
fig,axes = plt.subplots(nrows=4, ncols=1, figsize=(12,25))   
fig.subplots_adjust(bottom=0.2, left=0.2)

s = sns.barplot(data=BTM_head,x= 'count', y='cuisines',ax=axes[0])
s.set_title('Most served cuisines in BTM')
s.set_xlabel('Count of Restaurants provide the cuisines \n\n\nNorthIndian food served by most of the restaurant in BTM')
              
s1 = sns.barplot(data=BTM_tail,x= 'count', y='cuisines',ax=axes[1])
s1.set_xlabel('Count of Restaurants provide the cuisines \n\n\nLess serving food in BTM by Restaurant is Kashmiri food ')
s1.set_title('Less served cuisines in BTM')


s2 = sns.barplot(data=Kora_head,x= 'count', y='cuisines',ax=axes[2])
s2.set_xlabel('Count of Restaurants provide the cuisines \n\n\nNorthIndian food served by most of the restaurant in Koramangala')
s2.set_title('Most served cuisines in Koramangala 5th block')
              
s3 = sns.barplot(data=Kora_tail,x= 'count', y='cuisines',ax=axes[3])
s3.set_xlabel('Count of Restaurants provide the cuisines \n\n\nLess serving food in Koramangala by Restaurant is \nBihari,Geek, North Eastern,Awadhi food')
s3.set_title('Less served cuisines in Koramangala 5th block')


plt.tight_layout(pad=3.0)

##### For your restaurant you may go for most served cuisines may be the people like it more that's why most of the restaurants serving these cusines

##### Or you may go for less one, it might be having chances that they don't have much restaurant for these cuisines so there are some chances  for growth of you restaurant

##### 5. Best Location by frequency 

In [ ]:
city_freq = new_df.groupby('city')['city'].count().sort_values(ascending = False).head(5)
city_freq.plot(kind='pie',figsize=(6,6), shadow=True, autopct='%.1f%%')
plt.title('Best Location')
plt.tight_layout()

##### This is the pie chart of top Cities having more Restaurants

##### 6. Relation between Location and Rating


In [ ]:
# Grouping with city and ratings
relation_loc = new_df.groupby('city').agg({'ratings':'mean'}).sort_values(by='ratings', ascending = False).head(5)

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize= (20,20))
# s = sns.barplot(data=relation_loc, y=relation_loc.index, x= 'ratings', palette='winter')
relation_loc.plot(subplots=True,kind='pie',figsize=(6,6), legend = False,shadow=True, autopct='%.1f%%')

# s.set_ylabel('location')
# s.set_title('Relation between Location and Rating')

##### 7. Restaurant Type


In [ ]:
rest = new_df['resto_type'].value_counts().sort_values(ascending = False)

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize= (20,10))
s = sns.barplot(y=rest.index,x=rest.values)
s.set_xlabel('number of Availability')
s.set_ylabel('Restaurant Type')
s.set_title('Restaurant Types')

##### The most of the Restaurants types are Quick bites or Casual Dining whereas Meat shop and Pop up are less

##### 8. Gaussian Rest type and Rating


In [ ]:
rest_rating = new_df.groupby('resto_type').agg({'ratings':'mean'})

In [ ]:
sns.kdeplot(data=rest_rating)

##### By observation you can say that the mean of ratings as more in between 3 to 4

##### 9. Types of Services


In [ ]:
services = new_df['resto_type'].value_counts().sort_values(ascending = False).head(10)
services

In [ ]:
services.plot(kind='pie',figsize=(9,9), autopct='%.1f%%')
plt.title('Type of Services')
plt.tight_layout()

##### Here you can observed that Quick bites and Casual Dining are more oftenly provided by the Restaurants

##### 10. Relation between Type and Rating

In [ ]:
relation_type = new_df.groupby('resto_type').agg({'ratings':'mean'}).sort_values(by='ratings',ascending=False)

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize= (9,9))
s = sns.barplot(data= relation_type, x='ratings',y = relation_type.index)
s.set_xlabel('Ratings')
s.set_title('Relation between Restaurant type & Ratings')

##### Irani Cafe, Microbrewery, Club and Pub having more ratings then other type of Restaurants

##### 11. Cost of Restaurant


In [ ]:
approx_cost = new_df.groupby('Restro_name').agg({'approx_cost_for_two_people':'mean'}).sort_values(by= 'approx_cost_for_two_people',ascending = False).head(10)

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize= (7,4))
s = sns.barplot(data = approx_cost,y= approx_cost.index, x ='approx_cost_for_two_people')
s.set_xlabel('approx cost for two people')
s.set_title('Cost of Restaurant')

##### Some Restaurants in the city are more expensive for two person here, Le Cirque Signature - The Leela Palace takes approx Rs. 3000/person

##### 11.1 Most Expensive Restaurant's cuisines

In [ ]:
Rest_cui = new_df.loc[(new_df['Restro_name']=='Le Cirque Signature - The Leela Palace') | (new_df['Restro_name']=='Royal Afghan - ITC Windsor'),['Restro_name','cuisines']]
Rest_cui.drop_duplicates(inplace = True)

In [ ]:
Rest_cui = Rest_cui.groupby(['Restro_name','cuisines']).agg({'cuisines':'count'}).rename(columns = {'cuisines':'cui_count'}).reset_index()

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize= (7,4))
s = sns.barplot(data = Rest_cui,x= 'Restro_name', y ='cui_count',hue='cuisines')
s.set_ylabel("")

##### Now you'll decide whether you want to go for these restaurants or not

##### 12. No. of restaurants in a Location


In [ ]:
rest_city = new_df.groupby('city').agg({'Restro_name':'count'}).sort_values(by='Restro_name',ascending = False).head(10)

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize= (6,5))
s = sns.barplot(data=rest_city,x='Restro_name', y=rest_city.index)
s.set_xlabel('number of Restaurant')
s.set_ylabel('city')
s.set_title('Number of restaurants in a Location')

##### BTM having more Restaurants where as Bannerghatta Road having less Restaurants 

##### 13. Most famous restaurant chains in Bengaluru

In [ ]:
rest_vote = new_df.groupby('Restro_name').agg({'ratings':'sum','votes':'sum'})
vote = rest_vote.sort_values(by='votes', ascending=False).head(10)
rate = rest_vote.sort_values(by='ratings', ascending=False).head(10)

In [ ]:
fig,axes = plt.subplots(nrows=1, ncols=2, figsize=(15,5)) 
s = sns.barplot(data=rate,x= 'ratings', y=rate.index,ax=axes[0])
s.set_xlabel('Total Ratings')
s.set_ylabel('Restro_name')

s = sns.barplot(data=vote,x='votes', y=vote.index, ax=axes[1])
s.set_xlabel('Total Votes')
s.set_ylabel('Restro_name')
plt.suptitle('Most famous restaurant chains in Bengaluru')

plt.tight_layout()

##### The most Famous Restaurant Chain in the city are Empire Restaurant with highest votes and Truffles with highest ratings. But Onesta is on top with ratings as well as by votes 

#### Restaurant Search by location,Restaurant type,Cuisine type and budget

In [ ]:
recom = new_df.copy()

In [ ]:
recom.rename(columns={'Restro_full_address': 'Restaurant_Address', 'Restro_name':'Restaurant_Name','book_table':'Table_booking',
       'resto_type':'Restaurant_type'},inplace=True)

In [ ]:
def user_input():
    try:
        location = input('Enter the location : ')
        rest_type = input('Enter the restaurant type : ')
        cuisines = input('Enter the type of Cuisines : ')
        order_type = input('Do you want to order food online ? Yes for online & No for offline : ')
        people = int(input('Enter number of people'))
        max_budget = float(input('Enter your maximum budget'))

        return location,rest_type,cuisines,order_type,people,max_budget
    except:
        print('Something wrong with input')

In [ ]:
def zomato():
    
        location,rest_type,cuisines,order_type,people,max_budget = user_input()
    
    try:
        
        for_all_cost = (((recom.loc[(recom['city'] == location) & (recom['Restaurant_type']== rest_type) & 
                       (recom['cuisines']== cuisines) & (recom['online_order']== order_type),'approx_cost_for_two_people'])/2)*people)
#         print(for_all_cost)
        for i in for_all_cost:
          if i<=max_budget:
            
            print(f'\nApprox cost for {people} people is Rs.{for_all_cost.values} for below Restaurant\n')
            
            if order_type == 'No':    
                
                display(recom.loc[
                       (recom['city'] == location) & (recom['Restaurant_type']== rest_type) & 
                       (recom['cuisines']== cuisines) & (recom['online_order']== order_type),
                             ['Restaurant_Name','Restaurant_Address',
                              'Restaurant_type','approx_cost_for_two_people',
                              'ratings','votes','Table_booking','phone'
                             ]].sort_values(by = ['ratings','votes'],
                             ascending = False).set_index('Restaurant_Name').head(5), 'Thank You...!')
                
            else:
                display(recom.loc[(recom['city'] == location) & (recom['Restaurant_type']== rest_type) & 
                              (recom['cuisines']== cuisines) & (recom['online_order']== order_type),
                             ['Restaurant_Name','Restaurant_Address',
                              'Restaurant_type','approx_cost_for_two_people',
                              'ratings','votes']].sort_values(by = ['ratings','votes'],
                                ascending = False).set_index('Restaurant_Name').head(5), 'Thank You...!')
                
            break
    except:
        print('No Restaurant found')

zomato()